# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ariquemes,-9.9133,-63.0408,76.77,95,99,3.27,BR,1707606852
1,1,chonchi,-42.6128,-73.8092,62.31,69,11,1.10,CL,1707606852
2,2,wailua homesteads,22.0669,-159.3780,79.23,78,40,6.91,US,1707606852
3,3,forster,-32.1667,152.5167,74.37,79,88,12.21,AU,1707606853
4,4,merizo village,13.2658,144.6691,82.96,79,99,18.68,GU,1707606853


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
coordinates_df = city_data_df[["Lat", "Lng","City","Humidity"]]
coordinates_df = pd.DataFrame(coordinates_df)
coordinates_df 




,Lat,Lng,City,Humidity
0,-9.9133,-63.0408,ariquemes,95
1,-42.6128,-73.8092,chonchi,69
2,22.0669,-159.3780,wailua homesteads,78
3,-32.1667,152.5167,forster,79
4,13.2658,144.6691,merizo village,79
...,...,...,...,...
570,-28.5708,-70.7581,vallenar,37
571,-39.8142,-73.2459,valdivia,67
572,-23.3500,43.6667,toliara,79
573,10.4500,38.2000,bichena,68


In [20]:
# Create a map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    c="City",
    geo = True,
    alpha=0.5,
    scale= 1,
    tiles="OSM",
    size= "Humidity",
    frame_height = 500,
    frame_width = 800,
    title="City Latitude vs Longitude"
)

map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
best_cities_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 10), :]

# Drop any rows with null values
best_cities_df = best_cities_df.dropna()

# Display sample data
best_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,aioun,16.6614,-9.6149,78.13,6,0,9.24,MR,1707606860
57,57,codrington,-38.2667,141.9667,71.87,49,0,3.78,AU,1707606865
113,113,srivardhan,18.0333,73.0167,70.11,55,0,6.02,IN,1707606879
140,140,upwey,-37.9000,145.3333,79.39,50,0,3.00,AU,1707606884
143,143,saint-pierre,-21.3393,55.4781,76.68,78,0,6.91,RE,1707606885


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities_df[["City", "Country", "Lat", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
32,aioun,MR,16.6614,-9.6149,
57,codrington,AU,-38.2667,141.9667,
113,srivardhan,IN,18.0333,73.0167,
140,upwey,AU,-37.9000,145.3333,
143,saint-pierre,RE,-21.3393,55.4781,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places" 



    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
aioun - nearest hotel: فندق العيون
codrington - nearest hotel: No hotel found
srivardhan - nearest hotel: No hotel found
upwey - nearest hotel: Royal Hotel
saint-pierre - nearest hotel: Tropic Hotel
say - nearest hotel: No hotel found
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
gao - nearest hotel: No hotel found
mount gambier - nearest hotel: The Old Mount Gambier Gaol
al qadarif - nearest hotel: No hotel found
bairnsdale - nearest hotel: Main hotel
whakatane - nearest hotel: Whakatane Hotel


,City,Country,Lat,Lng,Hotel Name
32,aioun,MR,16.6614,-9.6149,فندق العيون
57,codrington,AU,-38.2667,141.9667,No hotel found
113,srivardhan,IN,18.0333,73.0167,No hotel found
140,upwey,AU,-37.9000,145.3333,Royal Hotel
143,saint-pierre,RE,-21.3393,55.4781,Tropic Hotel
266,say,NE,13.1007,2.3689,No hotel found
278,piton saint-leu,RE,-21.2196,55.3151,Appartement Piton Saint Leu
355,gao,ML,16.6362,1.6370,No hotel found
414,mount gambier,AU,-37.8333,140.7667,The Old Mount Gambier Gaol
461,al qadarif,SD,14.0349,35.3834,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    c="City",
    geo = True,
    alpha=0.5,
    scale= 1,
    tiles= "OSM",
    size= "Humidity",
    frame_height = 500,
    frame_width = 800,
    title="City Latitude vs Longitude"
)



# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City)